In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import re
from keras.models import load_model
from google.colab import files
uploaded = files.upload()
import pandas as pd
import io
import multiprocessing as mp
!pip install tensorflow scikeras
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV


from sklearn.preprocessing import LabelEncoder

mp.set_start_method('forkserver')

for file_name in uploaded.keys():
    print(f'Uploaded file: {file_name}')

data = pd.read_csv(io.BytesIO(uploaded[file_name]))

# Keeping only the neccessary columns
data = data[['text','sentiment']]

data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X)

embed_dim = 128
lstm_out = 196
def createmodel(): # Define the function before calling it
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
# print(model.summary())

labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

# Train the model
batch_size = 32
model = createmodel() # Now you can call the function
model.fit(X_train, Y_train, epochs=1, batch_size=batch_size, verbose=2)

# Save the model
model.save('sentiment_model.h5')

# Evaluate the model
score, acc = model.evaluate(X_test, Y_test, verbose=2, batch_size=batch_size)
print(score)
print(acc)
print(model.metrics_names)# Keeping only the neccessary columns
model = load_model('sentiment_model.h5')

# Function to preprocess new text data
def preprocess_text(text):
    text = text.lower()
    text = re.sub('[^a-zA-Z0-9\s]', '', text)
    text = text.replace('rt', ' ')
    return text

# Function to predict sentiment of new text
def predict_sentiment(text):
    text = preprocess_text(text)
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=X.shape[1])
    pred = model.predict(padded)
    return labelencoder.inverse_transform(pred.argmax(axis=1))[0]

new_text = "A lot of good things are happening. We are respected again throughout the world, and that's a great thing.@realDonaldTrump"
predicted_sentiment = predict_sentiment(new_text)
print(predicted_sentiment)

data = data[['text','sentiment']]

data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X)

embed_dim = 128
lstm_out = 196
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
# print(model.summary())

labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model.metrics_names)

def create_model(embed_dim=128, lstm_out=196, dropout_rate=0.2, optimizer='adam'):
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim, input_length=X.shape[1]))
    model.add(LSTM(lstm_out, dropout=dropout_rate, recurrent_dropout=dropout_rate))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Create KerasClassifier
model = KerasClassifier(build_fn=create_model,dropout_rate=0.2,embed_dim=128,lstm_out=196,verbose=2)

# Define parameter grid
param_grid = {
    'embed_dim': [128, 256],
    'lstm_out': [100, 196],
    'optimizer': ['adam', 'rmsprop'],
    'epochs': [1, 2],
    'batch_size': [32, 64]
}

# GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, Y_train)

# Print the best parameters and best score
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

Saving Sentiment .csv to Sentiment .csv
INFO: pip is looking at multiple versions of scikeras to determine which version is compatible with other requirements. This could take a while.
Uploaded file: Sentiment .csv
291/291 - 45s - loss: 0.8235 - accuracy: 0.6412 - 45s/epoch - 153ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


144/144 - 3s - loss: 0.7530 - accuracy: 0.6667 - 3s/epoch - 24ms/step
0.752994179725647
0.6666666865348816
['loss', 'accuracy']
1/1 [==============================] - 0s 299ms/step
Negative
291/291 - 45s - loss: 0.8221 - accuracy: 0.6462 - 45s/epoch - 154ms/step
144/144 - 3s - loss: 0.7667 - accuracy: 0.6737 - 3s/epoch - 20ms/step
0.7667195796966553
0.6736566424369812
['loss', 'accuracy']


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
